In [1]:
%load_ext autoreload
%autoreload 2
from glob import glob
import gc
import os
import sys
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
from func.utils import get_numeric_features, get_categorical_features, read_pkl_gzip, to_pkl_gzip, parallel_load_data, get_filename
from func.time_utils import date_add_days, date_add_times
from func.ml_utils import save_feature, get_cnt_feature, get_dummie_feature, get_label_feature, get_factorize_feature
from func.parallel_utils import get_parallel_arg_list
from joblib import delayed, Parallel
from itertools import combinations

In [ ]:
COLUMN_ID = 'TransactionID'
COLUMN_DT = 'TransactionDT'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_DT, COLUMN_TARGET, 'ProductCD']

train_paths = glob('../feature/eda_base/*_train.gz')
test_paths = glob('../feature/eda_base/*_test.gz')

train_paths = [path for path in train_paths 
               if path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
               or path.count('time_zone')
               or path.count('fill_')
               or path.count('bin_')
               or path.count('ProductCD')
              ]
test_paths = [path for path in test_paths 
               if path.count(COLUMN_DT) 
               or path.count(COLUMN_ID)
               or path.count(COLUMN_TARGET)
               or path.count('time_zone')
               or path.count('fill_')
               or path.count('bin_')
               or path.count('ProductCD')
              ]

df_train = parallel_load_data(train_paths)
df_test = parallel_load_data(test_paths)

In [3]:
use_cols = sorted([col for col in  df_train.columns if col not in COLUMNS_IGNORE])
print(len(use_cols))
cols_time   = ['time_zone']

cols_card       = sorted([col for col in use_cols if col.count('card') and not col.count('cnt')])
cols_addr       = sorted([col for col in use_cols if col.count('addr') and not col.count('cnt')])
cols_domain     = [col for col in df_train.columns if col.count('prefix') or col.count('_suffix') or col.count('_bin')]
cols_C          = sorted([col for col in use_cols if col.count('C') and not col.count('cnt')])
cols_V          = sorted([col for col in use_cols if col.count('_V') and not col.count('cnt')])
cols_M          = sorted([col for col in use_cols if col.count('M')])
cols_Amt        = sorted([col for col in use_cols if (col.count('Amt') or col.count('cents')) and not col.count('cnt')])

cols_cnt_card   = sorted([col for col in use_cols if col.count('card') and col.count('cnt')])
cols_cnt_addr   = sorted([col for col in use_cols if col.count('addr') and col.count('cnt')])
cols_cnt_C      = sorted([col for col in use_cols if col.count('C') and col.count('cnt')])
cols_cnt_V      = sorted([col for col in use_cols if col.count('V') and col.count('cnt')])
cols_cnt_Amt    = sorted([col for col in use_cols if (col.count('TransactionAmt') or col.count('cents')) and col.count('cnt')])

125


In [4]:
"""
複数のカラムを結合するにあたり、そのままの値だと文字数が多すぎて重いため、ラベリングする
"""
for col in tqdm(df_train.columns):
    df_train[col] = get_factorize_feature(df_train[[col]], [col]).values

100%|██████████| 129/129 [00:29<00:00,  9.15it/s]


In [5]:
"""
Normal. Not Count Feature
カラムは個別にして全組み合わせについて作るのは無理なので、個別に指定して作る
"""

# cols_all = cols_cnt_card+cols_cnt_addr+cols_cnt_C+cols_cnt_V+cols_cnt_Amt
# cols_all = cols_cnt_card+cols_cnt_addr+cols_cnt_C+cols_cnt_Amt

list_combi = []
#========================================================================
# Rule 1
#========================================================================
print(1)
for num in range(2, len(cols_C)+1, 1):
    combi_card = list(combinations(cols_card, num))
    for cards in combi_card:
        combi = list(cards) + cols_addr + cols_C + cols_V
        list_combi.append(combi)
        
#========================================================================
# Rule 2
#========================================================================
print(2)
for num in range(2, len(cols_C)+1, 1):
    combi_card = list(combinations(cols_card, num))
    for cards in combi_card:
        combi = list(cards) + cols_addr + cols_C
        list_combi.append(combi)
        
#========================================================================
# Rule 3
#========================================================================
print(3)
for num in range(2, len(cols_C)+1, 1):
    combi_card = list(combinations(cols_card, num))
    for cards in combi_card:
        combi = list(cards) + cols_addr + cols_V
        list_combi.append(combi)
        
#========================================================================
# Rule 4
#========================================================================
print(4)
for num in range(2, len(cols_C)+1, 1):
    combi_card = list(combinations(cols_card, num))
    for cards in combi_card:
        combi = list(cards) + cols_C
        list_combi.append(combi)
        
#========================================================================
# Rule 5
#========================================================================
print(5)
for num in range(2, len(cols_C)+1, 1):
    combi_card = list(combinations(cols_card, num))
    for cards in combi_card:
        combi = list(cards) + cols_V
        list_combi.append(combi)
        
#========================================================================
# Rule 6
#========================================================================
print(6)
for num in range(2, len(cols_C)+1, 1):
    combi_card = list(combinations(cols_card, num))
    for cards in combi_card:
        combi = list(cards) + cols_addr
        list_combi.append(combi)
        
#========================================================================
# Rule 7
#========================================================================
print(7)
for num in range(2, len(cols_C)+1, 1):
    combi_card = list(combinations(cols_card, num))
    for cards in combi_card:
        combi = list(cards) + cols_Amt
        list_combi.append(combi)

1
2
3
4
5
6
7


In [6]:
data = pd.concat([df_train, df_test], axis=0)

In [20]:
#========================================================================
# これはなに？
# 多くのカラムを使ってTEしたりLDAなどに使う文字列の結合featureを
# いちいち作るのは大変なので、ここでまとめて作っている
#========================================================================

train_length = df_train.shape[0]
prefix = '129'
dir_save = 'eda_base'

def parallel_concat_name(df, combi):
# df = data
# combi = list_combi[0]
# for i in range(1):
    list_concat = []
    combi = list(combi)
    list_name = [s.replace('fill__', '').replace('bin__', '') for s in combi]
    fname = '-'.join(list_name)
    name = f'combi{len(combi)}-{fname}'
    
    for val in tqdm(df[combi].values):
        val = [combi[i].replace('fill', 'f').replace('bin', 'b') + '_' + str(v) for i, v in enumerate(val)]
        concat_name = ' '.join(val)
        list_concat.append(concat_name)
    df[name] = list_concat
    train = df.iloc[:train_length][[name]]
    test  = df.iloc[train_length:][[name]]
    
    save_feature(train, prefix, dir_save, is_train=True, auto_type=False, list_ignore=COLUMNS_IGNORE)
    save_feature(test,  prefix, dir_save, is_train=False, auto_type=False, list_ignore=COLUMNS_IGNORE)

for i in tqdm(range(1, 21, 1)):
    Parallel(20)([delayed(parallel_concat_name)(data[combi], combi) for combi in list_combi[i*20:(i+1)*20]])

100%|██████████| 20/20 [3:25:11<00:00, 615.56s/it]


In [18]:
#========================================================================
# LDA用のALL COLUMN
#========================================================================
train_length = df_train.shape[0]
prefix = '129'
dir_save = 'eda_base'

combi = cols_cnt_card + cols_cnt_addr + cols_cnt_C + cols_cnt_V + cols_M + cols_domain

list_concat = []

for val in tqdm(data[combi].values):
    val = [combi[i].replace('fill', 'f').replace('bin', 'b') + '_' + str(v) for i, v in enumerate(val)]
    concat_name = ' '.join(val)
    list_concat.append(concat_name)
    
name = f'combi{len(combi)}_cnt_card-addr-domain-C-V-M'
data[name] = list_concat
train = data.iloc[:train_length][[name]]
test  = data.iloc[train_length:][[name]]

save_feature(train, prefix, dir_save, is_train=True, auto_type=False, list_ignore=COLUMNS_IGNORE)
save_feature(test,  prefix, dir_save, is_train=False, auto_type=False, list_ignore=COLUMNS_IGNORE)
del train, test
gc.collect()

(590540,) | combi92_cnt_card-addr-domain-C-V-M
(506691,) | combi92_cnt_card-addr-domain-C-V-M


43

In [ ]:
"""
Count Feature
カラムは個別にして全組み合わせについて作るのは無理なので、個別に指定して作る
"""

list_combi = []
#========================================================================
# Rule 1
#========================================================================
print(1)
for num in range(2, len(cols_cnt_C)+1, 1):
    combi_card = list(combinations(cols_cnt_card, num))
    for cards in combi_card:
        combi = list(cards) + cols_cnt_addr + cols_cnt_C + cols_cnt_V
        list_combi.append(combi)
        
#========================================================================
# Rule 2
#========================================================================
print(2)
for num in range(2, len(cols_cnt_C)+1, 1):
    combi_card = list(combinations(cols_cnt_card, num))
    for cards in combi_card:
        combi = list(cards) + cols_cnt_addr + cols_cnt_C
        list_combi.append(combi)
        
#========================================================================
# Rule 3
#========================================================================
print(3)
for num in range(2, len(cols_cnt_C)+1, 1):
    combi_card = list(combinations(cols_cnt_card, num))
    for cards in combi_card:
        combi = list(cards) + cols_cnt_addr + cols_cnt_V
        list_combi.append(combi)
        
#========================================================================
# Rule 4
#========================================================================
print(4)
for num in range(2, len(cols_cnt_C)+1, 1):
    combi_card = list(combinations(cols_cnt_card, num))
    for cards in combi_card:
        combi = list(cards) + cols_cnt_C
        list_combi.append(combi)
        
#========================================================================
# Rule 5
#========================================================================
print(5)
for num in range(2, len(cols_cnt_C)+1, 1):
    combi_card = list(combinations(cols_cnt_card, num))
    for cards in combi_card:
        combi = list(cards) + cols_cnt_V
        list_combi.append(combi)
        
#========================================================================
# Rule 6
#========================================================================
print(6)
for num in range(2, len(cols_cnt_C)+1, 1):
    combi_card = list(combinations(cols_cnt_card, num))
    for cards in combi_card:
        combi = list(cards) + cols_cnt_addr
        list_combi.append(combi)
        
#========================================================================
# Rule 7
#========================================================================
print(7)
for num in range(2, len(cols_cnt_C)+1, 1):
    combi_card = list(combinations(cols_cnt_card, num))
    for cards in combi_card:
        combi = list(cards) + cols_cnt_Amt
        list_combi.append(combi)

### EDA

In [ ]:
pd.set_option('max_rows', 500)
# df_train['cnt_sum'] = df_train[cols_cnt_card + cols_cnt_addr + cols_cnt_C].sum(axis=1)
df_train['cnt_sum'] = df_train[cols_cnt_card + cols_cnt_addr].sum(axis=1)
tmp = df_train.groupby(['cnt_sum'])[COLUMN_TARGET].agg({
    'mean': 'mean',
    'cnt': 'count',
})
tmp = tmp.sort_values(by='mean', ascending=False)
tmp[tmp['mean']<1]

In [52]:
lb9516 = read_pkl_gzip('../output/pred_result/20190909_2324__CV0-9467296784440689__all_preds.gz')
lb9516[COLUMN_ID] = lb9516[COLUMN_ID].astype('int')
df_train['_lb'] = lb9516.iloc[:, 1]

In [58]:
pd.set_option('max_columns', 500)
# df_train[df_train['cnt_sum']==9329487][COLUMNS_IGNORE + use_cols]
# df_train[df_train['cnt_sum']==12925646][COLUMNS_IGNORE + use_cols]
df_train[df_train['cnt_sum']==4534129][COLUMNS_IGNORE + use_cols]

,TransactionID,TransactionDT,isFraud,ProductCD,_lb,bin__Amt_DIV100,bin__Amt_DIV200,bin__Amt_DIV30,bin__Amt_DIV50,bin__C1,bin__C10,bin__C11,bin__C12,bin__C13,bin__C14,bin__C2,bin__C3,bin__C4,bin__C5,bin__C6,bin__C7,bin__C8,bin__C9,bin__TransactionAmt,bin__cents,cnt_sum,fill__M1,fill__M2,fill__M3,fill__M4,fill__M5,fill__M6,fill__M7,fill__M8,fill__M9,fill__P_emaildomain_bin,fill__P_emaildomain_prefix,fill__P_emaildomain_suffix,fill__R_emaildomain_bin,fill__R_emaildomain_prefix,fill__R_emaildomain_suffix,fill__addr1,fill__addr2,fill__card1,fill__card2,fill__card3,fill__card4,fill__card5,fill__card6,fill__cnt_C1,fill__cnt_C10,fill__cnt_C11,fill__cnt_C12,fill__cnt_C13,fill__cnt_C14,fill__cnt_C2,fill__cnt_C3,fill__cnt_C4,fill__cnt_C5,fill__cnt_C6,fill__cnt_C7,fill__cnt_C8,fill__cnt_C9,fill__cnt_P_emaildomain,fill__cnt_R_emaildomain,fill__cnt_TransactionAmt,fill__cnt_V127,fill__cnt_V128,fill__cnt_V130,fill__cnt_V131,fill__cnt_V133,fill__cnt_V156,fill__cnt_V165,fill__cnt_V187,fill__cnt_V2,fill__cnt_V201,fill__cnt_V243,fill__cnt_V258,fill__cnt_V259,fill__cnt_V265,fill__cnt_V267,fill__cnt_V281,fill__cnt_V282,fill__cnt_V283,fill__cnt_V29,fill__cnt_V294,fill__cnt_V3,fill__cnt_V306,fill__cnt_V307,fill__cnt_V308,fill__cnt_V310,fill__cnt_V312,fill__cnt_V313,fill__cnt_V314,fill__cnt_V315,fill__cnt_V317,fill__cnt_V318,fill__cnt_V320,fill__cnt_V37,fill__cnt_V38,fill__cnt_V4,fill__cnt_V44,fill__cnt_V45,fill__cnt_V48,fill__cnt_V49,fill__cnt_V5,fill__cnt_V53,fill__cnt_V54,fill__cnt_V6,fill__cnt_V61,fill__cnt_V62,fill__cnt_V67,fill__cnt_V7,fill__cnt_V70,fill__cnt_V76,fill__cnt_V78,fill__cnt_V83,fill__cnt_V87,fill__cnt_V90,fill__cnt_V91,fill__cnt_V94,fill__cnt_addr1,fill__cnt_addr2,fill__cnt_card1,fill__cnt_card2,fill__cnt_card3,fill__cnt_card4,fill__cnt_card5,fill__cnt_card6,fill__cnt_cents,time_zone
31095,3018095,771123,0,C,0.310877,98,98,28,48,"(157.0, 4685.0]","(15.0, 3257.0]","(117.0, 3188.0]","(6.0, 3188.0]","(171.0, 426.0]","(26.0, 82.0]","(149.0, 5691.0]","(-99999.001, 0.0]","(7.0, 2253.0]","(-99999.001, 0.0]","(92.0, 115.0]","(4.0, 2255.0]","(12.0, 3331.0]","(-99999.001, 0.0]","(2069.4, 2297.0]",0.99,4534129,#,#,#,M2,#,#,#,#,#,#,#,#,#,#,#,-100,-100,12616,490,150,visa,-100,credit,10856,10621,10769,9562,11066,11194,10808,1081893,10344,707011,10507,10638,10754,341564,163648,824070,90,675033,839537,747883,922414,1003636,939501,939225,820866,455805,821037,840073,840073,818499,840073,840073,1038094,401982,399663,88662,15,455805,15,15,15,15,15,905011,875936,900544,15,15,15,245823,245823,455805,245823,245823,245823,245823,455805,89995,89995,455805,89995,89995,89995,455805,89995,439725,903415,797446,918870,636219,634959,144184,131315,131315,468,70496,956845,719649,8806,267648,6651,22.0-23.5
155664,3142664,3222562,1,C,0.833944,3,3,3,3,"(18.0, 51.0]","(15.0, 3257.0]","(71.0, 89.0]","(6.0, 3188.0]","(91.0, 171.0]","(11.0, 26.0]","(149.0, 5691.0]","(-99999.001, 0.0]","(7.0, 2253.0]","(-99999.001, 0.0]","(32.0, 74.0]","(4.0, 2255.0]","(12.0, 3331.0]","(-99999.001, 0.0]","(1595.0, 1888.9]",0.03,4534129,#,#,#,M2,#,#,#,#,#,microsoft,hotmail,com,microsoft,hotmail,com,-100,-100,12616,490,150,visa,-100,credit,11068,10621,10896,9562,10830,10583,10808,1081893,10344,707011,11015,10638,10754,341564,85649,53166,3,675033,839537,747883,922414,1003636,939501,939225,12084,455805,20398,236794,222680,67308,184293,220568,1038094,401982,399663,652347,2,455805,1,1,1,658790,865652,905011,875936,900544,1,1,20,8032,12409,455805,803171,781678,556535,555462,455805,89995,89995,455805,89995,89995,89995,455805,89995,439725,903415,797446,1022,636219,634959,144184,131315,131315,468,70496,956845,719649,8806,267648,1347,6.5-10.0
161016,3148016,3385741,1,C,0.893148,51,151,11,1,"(18.0, 51.0]","(15.0, 3257.0]","(71.0, 89.0]","(6.0, 3188.0]","(91.0, 171.0]","(11.0, 26.0]","(149.0, 5691.0]","(-99999.001, 0.0]","(7.0, 2253.0]","(-99999.001, 0.0]","(32.0, 74.0]","(4.0, 2255.0]","(12.0, 3331.0]","(-99999.001, 0.0]","(6795.0, 7300.0]",0.51,4534129,#,#,#,M2,#,#,#,#,#,google,gmail,com,google,gmail